<a href="https://colab.research.google.com/github/manojach87/miscellaneous/blob/master/coingecko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.coingecko.com/api/documentations/v3#/operations/ping

In [ ]:
def readJsonLink(link):
  import urllib.request, json 
  with urllib.request.urlopen(link) as url:
      data = json.loads(url.read().decode())
      return data

In [ ]:
def getMarkets(url="https://api.coingecko.com/api/v3/coins/markets?vs_currency=USD&order=market_cap_desc&per_page=250&sparkline=false",pages=10):
  marketsData=list()
  for i in range(1,pages):
    #print(i)
    marketsData.append(readJsonLink(url+"&page="+str(i)))
  return marketsData
#marketsData

In [ ]:
def createDf(url,pages=10):
  import pandas as pd
  marketsData=getMarkets(url,pages=pages+1)
  marketsDf=pd.DataFrame()
  for i in range(0,len(marketsData)):
    #print(i)
    marketsDf=marketsDf.append(pd.DataFrame(marketsData[i]))
  marketsDf=marketsDf.reset_index()
  return marketsDf

In [ ]:
marketsDf=createDf(url="https://api.coingecko.com/api/v3/coins/markets?vs_currency=USD&order=market_cap_desc&per_page=250&sparkline=false",pages=28)

In [ ]:
#marketsDf[marketsDf.index==0]
marketsDf

,index,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,63080.000000,1.178632e+12,1,1.324732e+12,6.309478e+10,63705.00,62153.000000,2.940000,0.00466,-1.507376e+09,-0.12773,1.868399e+07,2.100000e+07,21000000.0,64805.00,-2.47155,2021-04-14T11:54:46.763Z,67.810000,9.310745e+04,2013-07-06T00:00:00.000Z,None,2021-04-16T03:56:20.062Z
1,1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,2504.830000,2.889361e+11,2,NaN,3.745754e+10,2544.30,2413.590000,35.040000,1.41860,3.172433e+09,1.11016,1.154874e+08,NaN,NaN,2544.30,-1.48109,2021-04-16T01:00:14.650Z,0.432979,5.788248e+05,2015-10-20T00:00:00.000Z,"{'times': 52.086480313239605, 'currency': 'btc...",2021-04-16T03:57:07.433Z
2,2,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/...,531.140000,8.175848e+10,3,9.022353e+10,5.327865e+09,554.88,521.820000,-14.078343,-2.58214,-2.499168e+09,-2.96610,1.545337e+08,1.705337e+08,170533651.9,610.06,-13.41817,2021-04-12T09:32:32.635Z,0.039818,1.326439e+06,2017-10-19T00:00:00.000Z,None,2021-04-16T03:56:59.336Z
3,3,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,1.740000,7.998640e+10,4,NaN,1.339644e+10,1.84,1.670000,-0.023405,-1.32776,-1.088368e+09,-1.34242,4.597332e+10,1.000000e+11,NaN,3.40,-48.81750,2018-01-07T00:00:00.000Z,0.002686,6.465346e+04,2014-05-22T00:00:00.000Z,None,2021-04-16T03:56:49.948Z
4,4,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.997999,4.688942e+10,5,NaN,1.592404e+11,1.00,0.998005,-0.002861,-0.28590,2.957864e+08,0.63482,4.698341e+10,4.698341e+10,NaN,1.32,-24.57086,2018-07-24T00:00:00.000Z,0.572521,7.431666e+01,2015-03-02T00:00:00.000Z,None,2021-04-16T03:07:17.281Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6719,219,gneiss,gneiss,Gneiss,https://assets.coingecko.com/coins/images/1335...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+09,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None
6720,220,franklin,fly,Franklin,https://assets.coingecko.com/coins/images/1481...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.700000e+09,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None
6721,221,fridaybeers,beer,FridayBeers,https://assets.coingecko.com/coins/images/1285...,NaN,NaN,None,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.200000e+04,32000.0,NaN,0.00000,None,NaN,0.000000e+00,None,None,2021-03-17T22:33:30.919Z
6722,222,the-apis,api,The APIS,https://assets.coingecko.com/coins/images/1405...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None


In [ ]:
len(marketsDf)

6686

In [ ]:
btcSupply=marketsDf[marketsDf.symbol=="btc"].circulating_supply[0]
btcSupply

18682068.0

In [ ]:
marketsDf["btc_relative_val"]=marketsDf.circulating_supply/btcSupply*marketsDf.current_price

In [ ]:
marketsDf.sort_values(by=["btc_relative_val"], ascending=[0], inplace=True)

In [ ]:
marketsDf[["symbol","btc_relative_val","current_price", "circulating_supply","market_cap"]][(marketsDf["btc_relative_val"]<20) & (marketsDf["btc_relative_val"]>10)]

,symbol,btc_relative_val,current_price,circulating_supply,market_cap
181,sfp,19.859240,3.430000,1.081667e+08,367124955.0
180,paid,19.651470,4.660000,7.878328e+07,367646469.0
182,woo,19.429467,0.875336,4.146781e+08,363753976.0
183,badger,19.366919,45.420000,7.965964e+06,359581165.0
185,bts,19.282041,0.132916,2.710196e+09,358268374.0
...,...,...,...,...,...
274,xcm,10.437659,0.643154,3.031888e+08,194997029.0
275,lto,10.238835,0.681934,2.805002e+08,190779240.0
276,mask,10.117666,17.320000,1.091333e+07,188961936.0
277,zmt,10.034137,4.670000,4.014099e+07,187495982.0


In [ ]:
mmDf=createDf(url="https://api.coingecko.com/api/v3/exchanges/binance/tickers?",pages=10)

In [ ]:
import pandas as pd
binanceDf=pd.DataFrame([ row for row in mmDf.tickers])[["coin_id","last"]]


In [ ]:
import plotly.express as px
fig = px.bar(marketsDf.head(200), x='symbol', y='btc_relative_val')
fig.show()

find coins that have not spiked historically beyond a certain threshold

In [ ]:
def getUnixTime(dt):
  import time
  import datetime
  if(dt is None):
    return int(time.time())
  else:
    d = datetime.date(dt[0],dt[1],dt[2])
    return int(time.mktime(d.timetuple()))

In [79]:
def getDate(timestamp):
  from datetime import datetime
  #try:
  return datetime.utcfromtimestamp(timestamp)
  #except ValueError:
  #  return int(timestamp)


In [80]:
(getUnixTime([2015,1,1]),getUnixTime(None))

(1420070400, 1618548890)

In [81]:
getDate(1618547586)

datetime.datetime(2021, 4, 16, 4, 33, 6)

In [95]:
def getHistory(coin="bitcoin",fromUnixTime=1420070400, toUnixTime=getUnixTime(None)):
  import pandas as pd
  apiurl="https://api.coingecko.com/api/v3/coins/"+coin+"/market_chart/range?vs_currency=usd&from="+str(fromUnixTime)+"&to="+str(toUnixTime)
  coinData=readJsonLink(apiurl)
  coinDf=pd.DataFrame(coinData["prices"], columns=["unixtime","price"])
  coinDf["date"]=coinDf.apply(lambda x : getDate(x["unixtime"]/1000),axis=1)
  coinDf=coinDf[["date","price"]]
  import plotly.express as px
  fig = px.line(coinDf, x="date", y="price", title=coin)
  fig.show()
  #return coinDf

In [96]:
getHistory("ethereum")
